In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from IPython.display import display

# install pkdb_analysis via:
#     git clone https://github.com/matthiaskoenig/pkdb_analysis
#     cd pkdb_analysis
#     pip install -e .

from pkdb_analysis import PKDB, PKData, PKFilter

# Filter data
A recurring task is to filter data for a certain question. E.g. to compare two groups, or get the subset of data for all healthy smokers.

We work again with our test data set and will filter various subsets from it.

In [2]:
test_data = PKDB.query()
TEST_HDF5 ="./test_data.hdf5"
test_data.to_hdf5(path=TEST_HDF5)

INFO *** Querying data ***
INFO http://0.0.0.0:8000/api/v1/interventions_analysis/?format=json&page_size=2000&normed=true
INFO http://0.0.0.0:8000/api/v1/interventions_analysis/?format=json&page_size=2000&normed=true&page=1
INFO http://0.0.0.0:8000/api/v1/individuals_analysis/?format=json&page_size=2000
INFO http://0.0.0.0:8000/api/v1/individuals_analysis/?format=json&page_size=2000&page=1
INFO http://0.0.0.0:8000/api/v1/individuals_analysis/?format=json&page_size=2000&page=2
INFO http://0.0.0.0:8000/api/v1/individuals_analysis/?format=json&page_size=2000&page=3
INFO http://0.0.0.0:8000/api/v1/individuals_analysis/?format=json&page_size=2000&page=4
INFO http://0.0.0.0:8000/api/v1/individuals_analysis/?format=json&page_size=2000&page=5
INFO http://0.0.0.0:8000/api/v1/individuals_analysis/?format=json&page_size=2000&page=6
INFO http://0.0.0.0:8000/api/v1/individuals_analysis/?format=json&page_size=2000&page=7
INFO http://0.0.0.0:8000/api/v1/individuals_analysis/?format=json&page_size=200

In [27]:
print(test_data)

------------------------------
PKData (140541721754704)
------------------------------
studies           221 
groups            606  ( 4857)
individuals      2903  (26380)
interventions     541  (  804)
outputs         19841  (19841)
timecourses      1376  ( 1376)
------------------------------


In [30]:
list(test_data.study_sids)[:10]

['PKDB00104',
 'PKDB00039',
 'PKDB00013',
 'PKDB00056',
 'PKDB00107',
 'PKDB00226',
 'PKDB00175',
 'PKDB00090',
 'PKDB00266',
 '4079279']

## Filter functions
The main principle for filtering `PKData` is by using the `filter_*` and `exclude_*` functionality.

A key principle are hereby filter functions which for a given DataFrame return a logical index.

Depending on which subset of information this should be applied the `groups`, `individuals`, `subjects` (groups and individuals), `outputs` and `timecourses`.

## Filter by `study_sid`
A first example is filtering by `study_sid`, i.e. we only want the subset of data from a single study.
An overview over the existing study sids in the dataset is available via

Filtering a subset of data works by providing filter/selection functions which select a subset of the data.
The filters are written on the `groups`, `individuals`

In [4]:
def is_PKDB99999(d):
    """Filter for specific study_sid. """
    return d.study_sid == "PKDB00198"

data = test_data.filter_intervention(is_PKDB99999)
print(data)

WARNING Concise DataFrames


------------------------------
PKData (140541887720592)
------------------------------
studies             1 
groups              4  (   36)
individuals        46  (  414)
interventions       1  (    1)
outputs            92  (   92)
timecourses         4  (    4)
------------------------------


The PKData now only contains data for the given study_sid:

In [5]:
print(data.study_sids)

{'PKDB00198'}


In [6]:
# for instance interventions
display(data.interventions)

,intervention_pk,study_sid,study_name,raw_pk,normed,name,route,form,application,time,...,substance,value,mean,median,min,max,sd,se,cv,unit
0,0,PKDB00198,Abernethy1982,377,True,paracetamol_iv,iv,solution,single dose,0.0,...,paracetamol,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram


One could also define this as a simple lambda function

In [7]:
data = test_data.filter_intervention(lambda d: d.study_sid == "PKDB00198")
print(data)

WARNING Concise DataFrames


------------------------------
PKData (140541885789776)
------------------------------
studies             1 
groups              4  (   36)
individuals        46  (  414)
interventions       1  (    1)
outputs            92  (   92)
timecourses         4  (    4)
------------------------------


## Concise data
All operations on `PKData` leave the data in a consistent manner. 
E.g. if an intervention is filtered out also all the outputs using this intervention are filtered out.
This behavior is controlled by the `concise` flag on most operations.

In [8]:
t1 = test_data.filter_intervention(is_PKDB99999)
t2 = test_data.filter_intervention(is_PKDB99999, concise=False)
print(t1)
print(t2)

WARNING Concise DataFrames


------------------------------
PKData (140541887727632)
------------------------------
studies             1 
groups              4  (   36)
individuals        46  (  414)
interventions       1  (    1)
outputs            92  (   92)
timecourses         4  (    4)
------------------------------
------------------------------
PKData (140541891193360)
------------------------------
studies           221 
groups            606  ( 4857)
individuals      2903  (26380)
interventions       1  (    1)
outputs         19841  (19841)
timecourses      1376  ( 1376)
------------------------------


In [9]:
# FIXME: only normed data
t1.interventions_mi

,study_sid,study_name,raw_pk,normed,name,route,form,application,time,time_end,...,substance,value,mean,median,min,max,sd,se,cv,unit
intervention_pk,,,,,,,,,,,,,,,,,,,,,
0,PKDB00198,Abernethy1982,377,True,paracetamol_iv,iv,solution,single dose,0.0,NaN,...,paracetamol,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram


In [10]:
t2.interventions_mi

,study_sid,study_name,raw_pk,normed,name,route,form,application,time,time_end,...,substance,value,mean,median,min,max,sd,se,cv,unit
intervention_pk,,,,,,,,,,,,,,,,,,,,,
0,PKDB00198,Abernethy1982,377,True,paracetamol_iv,iv,solution,single dose,0.0,NaN,...,paracetamol,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram


In [11]:
t2.outputs

,output_pk,intervention_pk,study_sid,study_name,group_pk,individual_pk,normed,calculated,tissue,time,...,substance,value,mean,median,min,max,sd,se,cv,unit
0,9333,0,PKDB00198,Abernethy1982,183,-1,True,False,plasma,NaN,...,paracetamol,NaN,2.5500,NaN,1.9900,3.4700,NaN,NaN,NaN,hour
1,9334,0,PKDB00198,Abernethy1982,183,-1,True,False,plasma,NaN,...,paracetamol,NaN,108.5000,NaN,62.2000,151.4000,NaN,NaN,NaN,liter
2,9335,0,PKDB00198,Abernethy1982,183,-1,True,False,plasma,NaN,...,paracetamol,NaN,0.8100,NaN,0.5300,1.3100,NaN,NaN,NaN,liter / kilogram
3,9336,0,PKDB00198,Abernethy1982,183,-1,True,False,plasma,NaN,...,paracetamol,NaN,29.0400,NaN,19.4400,38.7600,NaN,NaN,NaN,liter / hour
4,9337,0,PKDB00198,Abernethy1982,183,-1,True,False,plasma,NaN,...,paracetamol,NaN,0.2244,NaN,0.1452,0.3156,NaN,NaN,NaN,liter / hour / kilogram
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19836,50554,534,PKDB00114,Yue1991,-1,3643,True,True,plasma,NaN,...,normorphine,0.000071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram * hour / liter
19837,50555,534,PKDB00114,Yue1991,-1,3643,True,True,plasma,NaN,...,normorphine,0.000007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram / liter
19838,50556,534,PKDB00114,Yue1991,-1,3643,True,True,plasma,NaN,...,normorphine,0.001288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 / minute
19839,50557,534,PKDB00114,Yue1991,-1,3643,True,True,plasma,NaN,...,normorphine,8.966857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hour


## Query groups and individuals
### 2.1 Get data for groups with characteristica/keywords X
healthy=True, smoking=N, disease=None,
individual queries and combinations.


In [12]:
def is_healthy(d): 
    # healthy is reported and True
    return (d.measurement_type == "healthy") & (d.choice == "Y")

def disease(d):
    # any disease is reported
    return  d.measurement_type == "disease"

def smoking(d):
    # smoking status is curated for study (this could by Y/N/NR)
    return  d.measurement_type == "smoking"

def nonsmoker(d):
    # smoking is reported and no
    return smoking(d) & (d.choice == "N")

def smoker(d):
    # smoking is reported and yes
    return smoking(d) & (d.choice == "Y")

In [13]:
test_data = PKData.from_hdf5(TEST_HDF5)

`f_idx` can be a single function, or a list of functions. A list of functions are applied successively and is equivalent to "AND logic". "OR logic" can be directly applied on the index.

In [14]:
healthy_nonsmoker = test_data.filter_subject(f_idx=[is_healthy, nonsmoker])
print(healthy_nonsmoker)
healthy_nonsmoker.groups_mi

WARNING Concise DataFrames


------------------------------
PKData (140541914544208)
------------------------------
studies            90 
groups            151  ( 1507)
individuals      1075  (11006)
interventions     183  (  291)
outputs          7246  ( 7246)
timecourses       450  (  450)
------------------------------


study_sid     study_name group_name  group_count  \
group_pk characteristica_pk                                                     
183      3171                PKDB00198  Abernethy1982  obese men            7   
         3172                PKDB00198  Abernethy1982  obese men            7   
         3177                PKDB00198  Abernethy1982  obese men            7   
         3178                PKDB00198  Abernethy1982  obese men            7   
         3184                PKDB00198  Abernethy1982  obese men            7   
...                                ...            ...        ...          ...   
792      15378               PKDB00114        Yue1991        all            8   
         15379               PKDB00114        Yue1991        all            8   
         15380               PKDB00114        Yue1991        all            8   
         15381               PKDB00114        Yue1991        all            8   
         15382               PKDB00114        Yue1991        all            8   

                             group_parent_pk  count  \
group_pk characteristica_pk                           
183      3171                            182     42   
         3172                            182     42   
         3177                            182     21   
         3178                            182     21   
         3184                            182      7   
...                                      ...    ...   
792      15378                            -1      1   
         15379                            -1      7   
         15380                            -1      1   
         15381                            -1      8   
         15382                            -1      8   

                                        measurement_type        choice  \
group_pk characteristica_pk                                              
183      3171                                    species  homo sapiens   
         3172                                    healthy             Y   
         3177                              obesity index          None   
         3178                        weight (categorial)         obese   
         3184                                        sex             M   
...                                                  ...           ...   
792      15378                                   smoking             Y   
         15379                                   smoking             N   
         15380               smoking amount (cigarettes)          None   
         15381                                       age          None   
         15382                                    weight          None   

                            substance  value    mean  median    min  max  \
group_pk characteristica_pk                                                
183      3171                    None    NaN     NaN     NaN    NaN  NaN   
         3172                    None    NaN     NaN     NaN    NaN  NaN   
         3177                    None    NaN     NaN     NaN  133.0  NaN   
         3178                    None    NaN     NaN     NaN    NaN  NaN   
         3184                    None    NaN     NaN     NaN    NaN  NaN   
...                               ...    ...     ...     ...    ...  ...   
792      15378                   None    NaN     NaN     NaN    NaN  NaN   
         15379                   None    NaN     NaN     NaN    NaN  NaN   
         15380                   None    NaN  10.000     NaN    NaN  NaN   
         15381                   None    NaN  34.375     NaN    NaN  NaN   
         15382                   None    NaN  60.500     NaN    NaN  NaN   

                                sd  se  cv      unit  
group_pk characteristica_pk                           
183      3171                  NaN NaN NaN      None  
         3172                  NaN NaN NaN      None  
         3177                  NaN NaN NaN   percent  
         3178                  N

Often attributes are mixed for groups so we have to exclude the opposites.
In the example, the group `20` consists of 5 smokers and 1 nonsmoker. So for a subset of the group smoking is No.
We can exclude groups via

In [15]:
healthy_nonsmoker = test_data.filter_subject([is_healthy, nonsmoker]).exclude_subject([smoker])
print(healthy_nonsmoker)
display(healthy_nonsmoker.groups_mi)

WARNING Concise DataFrames
WARNING Concise DataFrames


------------------------------
PKData (140541914916368)
------------------------------
studies            77 
groups            114  ( 1081)
individuals       627  ( 5805)
interventions     157  (  258)
outputs          6012  ( 6012)
timecourses       389  (  389)
------------------------------


study_sid   study_name          group_name  \
group_pk characteristica_pk                                               
191      3444                PKDB00160  Adithan1982                 all   
         3445                PKDB00160  Adithan1982                 all   
         3446                PKDB00160  Adithan1982                 all   
         3447                PKDB00160  Adithan1982                 all   
         3448                PKDB00160  Adithan1982                 all   
...                                ...          ...                 ...   
789      15322               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   
         15325               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   
         15326               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   
         15332               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   
         15352               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   

                             group_count  group_parent_pk  count  \
group_pk characteristica_pk                                        
191      3444                         10               -1     10   
         3445                         10               -1     10   
         3446                         10               -1     10   
         3447                         10               -1     10   
         3448                         10               -1     10   
...                                  ...              ...    ...   
789      15322                        84              780    133   
         15325                        84              780    132   
         15326                        84              780      1   
         15332                        84              780    110   
         15352                        84              780     84   

                                measurement_type        choice substance  \
group_pk characteristica_pk                                                
191      3444                            species  homo sapiens      None   
         3445                                sex             M      None   
         3446                                age          None      None   
         3447                             weight          None      None   
         3448                            healthy             Y      None   
...                                          ...           ...       ...   
789      15322                            weight          None      None   
         15325               oral contraceptives             N      None   
         15326               oral contraceptives             Y      None   
         15332                               sex             M      None   
         15352                           smoking             N      None   

                             value  mean  median   min   max   sd    se  cv  \
group_pk characteristica_pk                                                   
191      3444                  NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         3445                  NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         3446                  NaN  21.2     NaN   NaN   NaN  NaN  0.47 NaN   
         3447                  NaN  61.6     NaN   NaN   NaN  NaN  1.94 NaN   
         3448                  NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
...                            ...   ...     ...   ...   ...  ...   ...  ..   
789      15322                 NaN  63.0     NaN  42.0  90.0  9.0   NaN NaN   
         15325                 NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         15326                 NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         15332                 NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         15352                 NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   

                                 unit  
group_pk characteristica_pk            
191      3444                    None  
         3445                  

In addition often combinations of attributes have to be used to find the correct subjects.
For instance a combination of `healthy` and reported `disease`

In [16]:
def is_healthy(d): 
    # healthy is reported and True
    return (d.measurement_type == "healthy") & (d.choice == "Y")

def disease(d):
    # any disease is reported
    return  d.measurement_type == "disease"

healthy1 = test_data.filter_subject(is_healthy)
healthy2 = test_data.exclude_subject(disease)
healthy3 = test_data.filter_subject(is_healthy).exclude_subject(disease)

print(healthy1)
print(healthy2)
print(healthy3)

WARNING Concise DataFrames
WARNING Concise DataFrames
WARNING Concise DataFrames
WARNING Concise DataFrames


------------------------------
PKData (140541898257104)
------------------------------
studies           198 
groups            334  ( 2935)
individuals      2164  (19140)
interventions     484  (  741)
outputs         17164  (17164)
timecourses      1220  ( 1220)
------------------------------
------------------------------
PKData (140541914805264)
------------------------------
studies           207 
groups            366  ( 3164)
individuals      2249  (19806)
interventions     503  (  760)
outputs         18416  (18416)
timecourses      1312  ( 1312)
------------------------------
------------------------------
PKData (140541914927504)
------------------------------
studies           197 
groups            334  ( 2935)
individuals      2144  (18991)
interventions     483  (  736)
outputs         17086  (17086)
timecourses      1220  ( 1220)
------------------------------


## 3 Query interventions
### 3.1 Get outputs/timecourses for intervention with substance
intervention with measurement_type "dosing" and substance "caffeine"

In [17]:
def dosing_and_caffeine(d):
    return ((d["measurement_type"]=="dosing") & (d["substance"]=="caffeine"))

In [18]:
test_data = PKData.from_hdf5(TEST_HDF5)

In [19]:
caffeine_data = test_data.filter_intervention(dosing_and_caffeine)

WARNING Concise DataFrames


In [20]:
print(caffeine_data)

------------------------------
PKData (140541760368528)
------------------------------
studies            63 
groups            102  (  954)
individuals       726  ( 6435)
interventions     110  (  207)
outputs          5747  ( 5747)
timecourses       330  (  330)
------------------------------


## 4 Query outputs/timecourses
### 4.1 query by measurement_type
filter all outputs with measurement_type auc_inf

In [21]:
def is_auc_inf(d):
    return (d["measurement_type"]=="auc_inf")  

test_data = PKData.from_hdf5(TEST_HDF5)

test_data = test_data.filter_output(is_auc_inf).delete_timecourses()
print(test_data)

WARNING Concise DataFrames
WARNING Concise DataFrames


------------------------------
PKData (140541761933392)
------------------------------
studies           155 
groups            200  ( 1713)
individuals       512  ( 4527)
interventions     336  (  529)
outputs          1877  ( 1877)
timecourses         0  (    0)
------------------------------


## 5 Other Query others
### 5.1 Complex
get clearance of codeine for all.h5 subjects, which have been phenotyped for cyp2d6. 


In [22]:
def is_cyp2d6_phenotyped(d):
    cyp2d6_phenotype_substances = ['spar/(2hspar+5hspar)', 'deb/4hdeb', 'dtf/dmt']
    return d["measurement_type"].isin(["metabolic phenotype", "metabolic ratio"]) & d["substance"].isin(cyp2d6_phenotype_substances)

def codeine_clearance(d):
    return (d["measurement_type"]=="clearance") & (d["substance"]=="codeine")                                                        

In [23]:
test_data = PKData.from_hdf5(TEST_HDF5)

phenotyped_data = test_data.filter_output(is_cyp2d6_phenotyped)

WARNING Concise DataFrames


In [24]:
test_data.groups = phenotyped_data.groups
test_data.individuals = phenotyped_data.individuals
test_data = test_data.filter_output(codeine_clearance).delete_timecourses()

WARNING Concise DataFrames
WARNING Concise DataFrames


In [25]:
print(test_data)

------------------------------
PKData (140541897941392)
------------------------------
studies            11 
groups             14  (  138)
individuals        13  (  135)
interventions      13  (   21)
outputs            30  (   30)
timecourses         0  (    0)
------------------------------
